In [1]:
##############################
## Bibliothèques nécéssaires 
##############################
import io
import os
import re
import json
import PyPDF2
import base64
import logging
import unicodedata
import numpy as np
import pandas as pd
from io import BytesIO
from openpyxl import load_workbook
from google.cloud import secretmanager
from datetime import datetime, timedelta
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from dateutil.relativedelta import relativedelta
from googleapiclient.http import MediaFileUpload
from google.oauth2.credentials import Credentials
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.http import MediaIoBaseDownload, MediaIoBaseUpload
import ast
from google.oauth2.credentials import Credentials as Credentials_user
from google.oauth2.service_account import Credentials as Credentials_sa
from googleapiclient.discovery import build




FOLDER_NAME, FILENAME = "NE PAS SUPPRIMER", "processed_ids.json"
TARGET_DRIVE_FOLDER_ID = "13_F3h2Elh4JCW50JvPtvX19l8ZstmIny"


## Authentification

def get_creds_mail():
    gcp_secret_client = secretmanager.SecretManagerServiceClient()
    secret_resource_name = "projects/437392419716/secrets/o2c-ca-token/versions/1"
    response = gcp_secret_client.access_secret_version(request={"name": secret_resource_name})
    token = ast.literal_eval(response.payload.data.decode("UTF-8"))
    return Credentials_user.from_authorized_user_info(token)

def get_creds():
    gcp_secret_client = secretmanager.SecretManagerServiceClient()
    secret_resource_name = "projects/437392419716/secrets/sa_o2c_relance_client_private_key/versions/1"
    response = gcp_secret_client.access_secret_version(request={"name": secret_resource_name})
    token = ast.literal_eval(response.payload.data.decode("UTF-8"))
    return Credentials_sa.from_service_account_info(token)


CREDS = get_creds()
CREDS_MAIL = get_creds_mail()

#  Initialisation des services Drive et Gmail
try:
    drive_service = build('drive', 'v3', credentials=CREDS, cache_discovery=False)
    gmail_service = build('gmail', 'v1', credentials=CREDS_MAIL, cache_discovery=False)
except Exception as e:
    logging.error(f"Erreur lors de la création des services Google API : {e}")
    exit(1)





#################################
## fichier des log pour debugging
#################################

logging.basicConfig(
    level=logging.INFO,
    filename='extraction.log',
    filemode='a',
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logging.info("Début du traitement...")





#############################
## 1ère partie  fonctions  ##
#############################

# nettoyer les noms pour une comparaison optimale
def normaliser_nom(nom):
    if pd.isnull(nom):
        return None
    nom = str(nom)
    nom = unicodedata.normalize('NFD', nom).encode('ascii', 'ignore').decode("utf-8")
    nom = nom.upper()
    nom = re.sub(r'[^\w\s]', ' ', nom)
    nom = ' '.join(nom.split())
    return nom



# telecharger les fichiers sur le drive 
def download_file_from_drive(service, file_id):
    file_stream = io.BytesIO()
    downloader = MediaIoBaseDownload(file_stream, service.files().get_media(fileId=file_id))
    done = False
    while not done:
        _, done = downloader.next_chunk()
    file_stream.seek(0)
    return file_stream



#Recherche un fichier sur Google Drive par son nom et son type MIME, dans un dossier donné si précisé.
#Si le fichier existe, retourne son ID ; sinon, le crée et retourne l'ID du fichier créé.
def get_or_create(service, name, mime, parent=None, content=None):
    q = f"name='{name}' and mimeType='{mime}'" + (f" and '{parent}' in parents" if parent else "")
    r = service.files().list(
        q=q,
        spaces='drive',
        fields='files(id)',
        supportsAllDrives=True,               
        includeItemsFromAllDrives=True        
    ).execute().get('files', [])

    if r:
        return r[0]['id']

    meta = {'name': name, 'mimeType': mime} | ({'parents': [parent]} if parent else {})
    media = MediaIoBaseUpload(BytesIO(content.encode()) if content else None, mimetype="application/json") if content else None
    return service.files().create(
        body=meta,
        media_body=media,
        fields='id',
        supportsAllDrives=True                
    ).execute()['id']



#Charge l'état depuis le fichier Drive : les IDs déjà traités et la dernière date de traitement.
#Si le fichier est vide, retourne un état initial avec un set vide et une date par défaut.

def load_state(service, fid):
    req, buf = service.files().get_media(fileId=fid), BytesIO()
    downloader = MediaIoBaseDownload(buf, req)
    done = False
    while not done:
        _, done = downloader.next_chunk()
    buf.seek(0)
    content = buf.read()
    if not content:
        return set(), "2025/04/29"  #  Date de départ si fichier vide

    data = json.loads(content)
    ids = set(data.get("processed_ids", []))
    last_date = data.get("last_processed_date", "2025/04/29")
    return ids, last_date



 
#Sauvegarde l'état : les IDs déjà traités et la dernière date de traitement.
#Écrase le contenu du fichier sur Drive. 
def save_state(service, fid, ids, last_date):
    data = {
        "processed_ids": list(ids),
        "last_processed_date": last_date
    }
    media = MediaIoBaseUpload(BytesIO(json.dumps(data).encode()), mimetype="application/json")
    service.files().update(fileId=fid, media_body=media).execute()



# Force la mise à jour de la date dans le fichier  sur Drive 
def force_update_state(service, file_id, forced_date):
    data = {
        "processed_ids": [],  
        "last_processed_date": forced_date
    }
    media = MediaIoBaseUpload(BytesIO(json.dumps(data).encode()), mimetype="application/json")
    service.files().update(fileId=file_id, media_body=media).execute()
    logging.info(f"Date forcée à : {forced_date}")




def get_csvs_from_gmail(service, query, ids):
    results = service.users().messages().list(userId="me", q=query).execute()
    new_ids, csvs = set(), []
    for msg in results.get("messages", []):
        mid = msg["id"]
        if mid in ids: continue
        msg_data = service.users().messages().get(userId="me", id=mid).execute()
        for p in msg_data["payload"].get("parts", []):
            if p.get("filename", "").endswith(".csv") and "attachmentId" in p["body"]:
                data = service.users().messages().attachments().get(userId="me", messageId=mid, id=p["body"]["attachmentId"]).execute()
                s = BytesIO(base64.urlsafe_b64decode(data["data"])); s.name = p["filename"]
                csvs.append({"stream": s, "message_id": mid}); new_ids.add(mid)
    return csvs, new_ids




# Téléchargement des fichiers sur le drive  et transformation en dataframe
BD_Couleur=pd.read_excel(download_file_from_drive(drive_service, '1y_eu0l2ZrqQH7sWn6-HOlE_FHyL5afJA'), engine='openpyxl')
BD_VIOLETTE=pd.read_excel(download_file_from_drive(drive_service, '1K9KMDL2QyCUprcWIWxZz_E9vtomxm4z7'), engine='openpyxl')
BD_ROUGE=pd.read_excel(download_file_from_drive(drive_service, '1riHpyh_wBvIOjGSB7ykF2TmpDNFcnvUr'), engine='openpyxl')
BD_ORANGE=pd.read_excel(download_file_from_drive(drive_service, '16BsnVrVZXANsNJ8vR2A1x1p1Pwgb5m2X'), engine='openpyxl')


# Création / récupération du dossier et du fichier JSON
folder_id = get_or_create(drive_service, FOLDER_NAME, 'application/vnd.google-apps.folder')
file_id = get_or_create(drive_service, FILENAME, 'application/json', folder_id, content='{"processed_ids": [], "last_processed_date": "2025/01/01"}')


###################################################################################################
###################################################################################################
## Pour forcer la date 
forcer_date = True  # Mettre à True pour forcer la date

if forcer_date:
    force_update_state(drive_service, file_id, "2025/05/08")  # mettre la date voulue ici
##
####################################################################################################
####################################################################################################



# Charger l'état (IDs + date)
try:
    ids, last_processed_date = load_state(drive_service, file_id)
    logging.info(f"{len(ids)} IDs déjà traités. Dernière date de traitement : {last_processed_date}")
except Exception as e:
    logging.error(f"Erreur lors du chargement de l'état depuis Drive : {e}")
    exit(1)


# Construire la requête avec la dernière date
query = f"from:automail@epay.de filename:csv after:{last_processed_date}"
#query = f"from:automail@epay.de filename:csv after:{last_processed_date}"
#query = f"in:anywhere from:gaetan_djambissie_nana@carrefour.com filename:csv after:{last_processed_date}"



# Chercher les nouveaux CSVs depuis Gmail
try:
    csvs, new_ids = get_csvs_from_gmail(gmail_service, query, ids)
    logging.info(f"{len(csvs)} nouveaux fichiers CSV depuis Gmail.")
except Exception as e:
    logging.error(f"Erreur lors de la récupération des CSV depuis Gmail : {e}")
    exit(1)


# Si des nouveaux mails, mettre à jour l'état
if new_ids:
    ids.update(new_ids)
    today_str = datetime.now().strftime("%Y/%m/%d")  
    save_state(drive_service, file_id, ids, today_str)
    logging.info(f"État mis à jour avec {len(new_ids)} nouveaux IDs et nouvelle date : {today_str}")
else:
    logging.info("Aucun nouveau mail à traiter. Pas de mise à jour nécessaire.")

# FILTRE les fichiers dont le nom contient 'DailySCC_Orders'
matching_csvs = [c for c in csvs if "DailySCC_Orders" in (c['stream'].name or "")]
logging.info(f"{len(matching_csvs)} fichiers correspondent à 'DailySCC_Orders'.")


###############################################################
## TRAITEMENT DES FICHIERS DAILY 
###############################################################

for c in matching_csvs:
    stream = c['stream']
    msg_id = c['message_id']
    stream.seek(0)

    filename = stream.name

        # Enlève l'extension convertit en date et ajoute 1 jour
    if '.' in filename:
        date_str = filename.split('_')[-1].split('.')[0].strip()
    else:
        date_str = filename.split('_')[-1].strip()

    date_obj = datetime.strptime(date_str, '%Y%m%d')  
    next_day = date_obj + timedelta(days=1)          
    sem = next_day.strftime('%d-%m-%Y')              

    try:
        df = pd.read_csv(stream, encoding='ISO-8859-1', sep=';')
        logging.info(f"Fichier : {stream.name} - {len(df)} lignes")

        colonnes_a_nettoyer = [
        "Montant des cartes",
        "Remise",
        "Frais livraison HT",
        "Frais additionnels",
        "TVA totale",
        "Montant total"
         ]
        
        for col in colonnes_a_nettoyer:
            df.loc[:, col] = pd.to_numeric(
                df[col].astype(str).str.replace(",", "").str.strip(),
                errors='coerce'
            ) 


        weekly = df[df["Commande API"] == "Non"].copy()
        

        
        # Séparation CB vs sans CB
        weekly_CB = weekly[weekly["Moyen de paiement"] == "Carte bleue"]
        weekly_sans_CB = weekly[weekly["Moyen de paiement"] != "Carte bleue"].copy()

        # nettoyage des différents DataFrames
        BD_Couleur['Nom Société clean'] = BD_Couleur['Nom Société'].apply(normaliser_nom)
        weekly_sans_CB['Nom Société clean'] = weekly_sans_CB['Nom Société'].apply(normaliser_nom)
        BD_VIOLETTE['Nom Société clean'] = BD_VIOLETTE['Nom Société'].apply(normaliser_nom)
        BD_ROUGE['Nom Société clean'] = BD_ROUGE['Nom Société'].apply(normaliser_nom)
        BD_ORANGE['Nom Société clean'] = BD_ORANGE['Nom Société'].apply(normaliser_nom)
        
        # Extraction des HYPER
        noms_hyper = BD_Couleur[BD_Couleur["type de facture"] == "HYPER"]['Nom Société clean'].unique()
        weekly_hyper = weekly_sans_CB[weekly_sans_CB['Nom Société clean'].isin(noms_hyper)].copy()

        # Ajout du compte comptable et du centre de coût aux HYPER 
        compte_dict = BD_VIOLETTE.set_index('Nom Société clean')['Compte comptable-D SAP'].to_dict()
        weekly_hyper['Compte comptable-D SAP'] = weekly_hyper['Nom Société clean'].map(compte_dict)

        centre_cout_dict = BD_VIOLETTE.set_index('Nom Société clean')['Centre de coût-D'].to_dict()
        weekly_hyper['Centre de coût-D'] = weekly_hyper['Nom Société clean'].map(centre_cout_dict)

        # tri par rapport au nom de société puis si identiques par rapport au numero commande 
        weekly_hyper = weekly_hyper.sort_values(
        by=["Nom Société", "Numero Commande"],
        key=lambda col: col.str.casefold() if col.name == "Nom Société" else col)


        # Extraction des Oranges
        societes_orange = BD_ORANGE['Nom Société clean'].unique()
        weekly_Orange = weekly_sans_CB[weekly_sans_CB['Nom Société clean'].isin(societes_orange)].copy()
        # weekly_Orange['Date'] = pd.to_datetime(weekly_Orange['Date']).dt.date
        weekly_Orange['Date'] = pd.to_datetime(weekly_Orange['Date'], errors='coerce').dt.strftime('%d.%m.%Y')

        # Convertir les montants en nombre si ce n’est pas déjà fait
        weekly_Orange['Montant total'] = pd.to_numeric(weekly_Orange['Montant total'], errors='coerce')

        # Formater avec 2 décimales et remplacer le point par une virgule
        weekly_Orange['Montant total'] = weekly_Orange['Montant total'].apply(
        lambda x: f"{x:.2f}".replace('.', ',') if pd.notna(x) else '')

        # Ajout des colonnes BU-D   CLIENT acheteur aux ORANGES 
        BU_dict = BD_ORANGE.set_index('Nom Société clean')['BU-D'].to_dict()
        weekly_Orange['BU-D'] = weekly_Orange['Nom Société clean'].map(BU_dict)

        cl_dict = BD_ORANGE.set_index('Nom Société clean')['CLIENT acheteur'].to_dict()
        weekly_Orange['CLIENT acheteur'] = weekly_Orange['Nom Société clean'].map(cl_dict)
        
        weekly_Orange = weekly_Orange.sort_values(by='Nom Société')
      

        fichier_weekly_Orange = weekly_Orange.drop(columns=["Nom Société clean"])


        # Extraction des  Rouges
        societes_rouge = BD_ROUGE['Nom Société clean'].unique()
        weekly_Rouge = weekly_sans_CB[weekly_sans_CB['Nom Société clean'].isin(societes_rouge)].copy()
        # weekly_Rouge['Date'] = pd.to_datetime(weekly_Rouge['Date']).dt.date
        weekly_Rouge['Date'] = pd.to_datetime(weekly_Rouge['Date'], errors='coerce').dt.strftime('%d.%m.%Y')

        # Convertir les montants en nombre si ce n’est pas déjà fait
        weekly_Rouge['Montant total'] = pd.to_numeric(weekly_Rouge['Montant total'], errors='coerce')

        # Formater avec 2 décimales et remplacer le point par une virgule
        weekly_Rouge['Montant total'] = weekly_Rouge['Montant total'].apply(
        lambda x: f"{x:.2f}".replace('.', ',') if pd.notna(x) else '')



        # Ajout des colonnes CLIENT acheteur  CP (centre profit)    CC (centre coût) aux ROUGES 
        client_dict = BD_ROUGE.set_index('Nom Société clean')['CLIENT acheteur'].to_dict()
        weekly_Rouge['CLIENT acheteur'] = weekly_Rouge['Nom Société clean'].map(client_dict)

        cp_dict = BD_ROUGE.set_index('Nom Société clean')['CP (centre profit)'].to_dict()
        weekly_Rouge['CP (centre profit)'] = weekly_Rouge['Nom Société clean'].map(cp_dict)

        cc_dict = BD_ROUGE.set_index('Nom Société clean')['CC (centre coût)'].to_dict()
        weekly_Rouge['CC (centre coût)'] = weekly_Rouge['Nom Société clean'].map(cc_dict)

        weekly_Rouge = weekly_Rouge.sort_values(by='Nom Société')

       


        fichier_weekly_Rouge = weekly_Rouge.drop(columns=["Nom Société clean"])


        # 3. Extraire la liste des sociétés connues
        societes_connues = BD_Couleur['Nom Société clean'].unique()
        weekly_NOIR = weekly_sans_CB[~weekly_sans_CB['Nom Société clean'].isin(societes_connues)].copy()

        # tri par rapport au nom de société puis si identiques par rapport au numero commande 
        weekly_NOIR = weekly_NOIR.sort_values(
        by=["Nom Société", "Numero Commande"],
        key=lambda col: col.str.casefold() if col.name == "Nom Société" else col)
    

    

        # Etape de  remplissage des differents fichiers 
        # entete pour les fichiers de  lignes verte et violette 
        colonnes = [
            "Date comptable","Ledger","Code Société", "Centre de profit","Domaine d'activité","Centre de coût","Élément d'OTP","Compte comptable","Débit/Crédit","Montant",
            "Description écriture","Code TVA", "Code Business Partner","Code CGS","Code Société SL partenaire","Centre de profit SL partenaire","Clé de lettrage",
            "Nature de retraitement","Code flux","Code AFB","Millésime","Nature Fiscale","Nature Reprise","Clé Complémentaire","Affectation"
        ]

    # Remplissage  des OD VERTE
        compte_debit =  2830200000
        compte_credit=  2109000000

        maquette_data = []
        for sens in ['D', 'C']:
            for i in range(len(weekly_CB)):
                row = {col: '' for col in colonnes}
                date_val = weekly_CB.iloc[i]['Date']
                if pd.notna(date_val):
                    date_dt = pd.to_datetime(date_val, errors='coerce')
                    row['Date comptable'] =  date_dt.strftime("%d/%m/%Y")
                else:
                    row['Date comptable'] = ''
                
                row['Montant'] = weekly_CB.iloc[i]['Montant total']
                row['Description écriture'] = f"{int(weekly_CB.iloc[i]['Numero Commande'])} CSPF  CB 28300120"
                row['Débit/Crédit'] = sens
                row['Code Société'] = 1386
                row['Centre de profit'] =  '33A840' 
                
                # Compte comptable dépend du sens
                if sens == 'D':
                    row['Compte comptable'] = compte_debit

                    if pd.notna(date_val):
                        date_dt = pd.to_datetime(date_val, errors='coerce')
                        row['Clé de lettrage'] = date_dt.strftime("%Y-%m-%d")
                        row['Affectation'] = date_dt.strftime("%Y-%m-%d")
                    else:
                        row['Clé de lettrage'] = ''
                        row['Affectation'] = ''


                else:
                    row['Compte comptable'] = compte_credit
                    row['Clé de lettrage'] = 'SCC'
                    row['Affectation'] = 'SCC'

                maquette_data.append(row)

    # Remplissage des OD VIOLETTE
        credit_compteC=  2109000000
        maquette_data1 = []

        for sens in ['D', 'C']:
            for i in range(len(weekly_hyper)):
                row = {col: '' for col in colonnes}
                date_val = weekly_hyper.iloc[i]['Date']
                if pd.notna(date_val):
                    date_dt = pd.to_datetime(date_val, errors='coerce')
                    row['Date comptable'] = date_dt.strftime("%d/%m/%Y")
                else:
                    row['Date comptable'] = ''
                
                row['Montant'] = weekly_hyper.iloc[i]['Montant total']
                row['Description écriture'] = f"{int(weekly_hyper.iloc[i]['Numero Commande'])} CSPF  CB CDE"
                row['Débit/Crédit'] = sens
                row['Code Société'] = 1386
                
                if sens == 'C':
                    row['Compte comptable'] = credit_compteC
                    row['Clé de lettrage'] = 'SCC'
                    row['Affectation'] = 'SCC'
                    row['Centre de profit'] =  '33A840' 
                else:
                    centre_cout = weekly_hyper.iloc[i]['Centre de coût-D']
                    row['Centre de coût'] =  centre_cout 
                    compte_debit_val = weekly_hyper.iloc[i]['Compte comptable-D SAP']
                    row['Compte comptable'] = compte_debit_val

                #Code TVA selon le compte
                    if compte_debit_val == 6150300000:
                        row['Code TVA'] = 'A0'
                    elif compte_debit_val == 6141000000:
                        row['Code TVA'] = 'V0'
                    else:
                        row['Code TVA'] = ''
            
                maquette_data1.append(row)

    # Remplissage des IRI 
        colonnes3 = [
        "Code évenement","Date comptable (jj/mm/aaaa)","Code de distribution","Business Unit", "Code client","Numéro de pièce","Type de pièce","Code comptable","Opérating unit","Code département","Montant TTC",
        "Date d'échéance(jj/mm/aaaa)","Type de transaction", "Montant TVA","Taux de TVA","Commentaire","Clé de lettrage", "Mode de paiement", "Adresse Mail pour envoie de la facture"
        ]

        maquette_data2 = []

        for i in range(len(weekly_NOIR)):
            row = {col: '' for col in colonnes3}
            date_val = weekly_NOIR.iloc[i]['Date']

            if pd.notna(date_val):
                date_dt = pd.to_datetime(date_val, errors='coerce')
                row['Date comptable (jj/mm/aaaa)'] = date_dt.strftime("%d/%m/%Y")
            else:
                row['Date comptable (jj/mm/aaaa)'] = ''


            row['Code évenement'] = 'IRI'
            row['Code de distribution'] = 'DC-RCVBL'
            row['Business Unit'] = 'FR246'
            row["Code client"] = weekly_NOIR.iloc[i]["Code client"]
            row["Numéro de pièce"] = weekly_NOIR.iloc[i]["Numero Commande"]
            row['Type de pièce'] = 'FACTU'
            row['Code comptable'] = '21SCC' 
            row['Opérating unit'] = 'FRA903' 
            row['Code département'] = '03900' 
            row["Montant TTC"] = weekly_NOIR.iloc[i]["Montant total"]
            row['Type de transaction'] = 'SVSE'
            row['Montant TVA'] = 0
            row['Taux de TVA'] = 'ZER'
            row["Commentaire"] = weekly_NOIR.iloc[i]["Numero Commande"]
            row['Clé de lettrage'] = 'SCC'   
            
            # Ajout d'1 mois à la date pour Date d'échéance
            if pd.notna(date_val):
                date_dt = pd.to_datetime(date_val, errors='coerce')
                if pd.notna(date_dt):
                    echeance = date_dt + relativedelta(months=1)
                    row["Date d'échéance(jj/mm/aaaa)"] = echeance.strftime("%d/%m/%Y")
                else:
                    row["Date d'échéance(jj/mm/aaaa)"] = ''

            maquette_data2.append(row)

        # Créer un dossier  pour acceuillir tout les  fichier  (VERT , VIOLET , IRI)
        folder_vert_metadata = {
            "name": f"fichiers_du_{sem}",
            "mimeType": "application/vnd.google-apps.folder",
            "parents": [TARGET_DRIVE_FOLDER_ID]
        }
        folder = drive_service.files().create(
            body=folder_vert_metadata,
            fields="id",
            supportsAllDrives=True
        ).execute()

        # Générer fichier VERT (weekly_CB)
        maquette_finale_verte = pd.DataFrame(maquette_data, columns=colonnes)
        buffer_verte = BytesIO()
        maquette_finale_verte.to_excel(buffer_verte, index=False)
        buffer_verte.seek(0)

        file_metadata_verte = {
            "name": f"OD LIBRES O2C-CB {sem}.xlsx",
            "parents": [folder['id']],
            "mimeType": "application/vnd.openxmlformats-officedocument.spreadsheetml.sheet"
        }
        media_verte = MediaIoBaseUpload(buffer_verte, mimetype="application/vnd.openxmlformats-officedocument.spreadsheetml.sheet")
        drive_service.files().create(
            body=file_metadata_verte,
            media_body=media_verte,
            fields="id",
            supportsAllDrives=True
        ).execute()

        logging.info(f"Fichier VERT généré : OD LIBRES O2C-VERTE {sem}.xlsx")

        # Générer fichier VIOLET (DailySCC_hyper) 
        maquette_finale_violet = pd.DataFrame(maquette_data1, columns=colonnes)
        buffer_violet = BytesIO()
        maquette_finale_violet.to_excel(buffer_violet, index=False)
        buffer_violet.seek(0)

        file_metadata_violet = {
            "name": f"OD LIBRES O2C-HYPER {sem}.xlsx",
            "parents": [folder['id']],
            "mimeType": "application/vnd.openxmlformats-officedocument.spreadsheetml.sheet"
        }
        media_violet = MediaIoBaseUpload(buffer_violet, mimetype="application/vnd.openxmlformats-officedocument.spreadsheetml.sheet")
        drive_service.files().create(
            body=file_metadata_violet,
            media_body=media_violet,
            fields="id",
            supportsAllDrives=True
        ).execute()

        logging.info(f"Fichier VIOLET généré : OD LIBRES O2C-VIOLET {sem}.xlsx")

        #  Générer fichier IRI 
        maquette_finale_IRI = pd.DataFrame(maquette_data2, columns=colonnes3)
        buffer_IRI = BytesIO()
        maquette_finale_IRI.to_excel(buffer_IRI, index=False)
        buffer_IRI.seek(0)

        file_metadata_IRI = {
            "name": f"IRI_CRP_{sem}.xlsx",
            "parents": [folder['id']],
            "mimeType": "application/vnd.openxmlformats-officedocument.spreadsheetml.sheet"
        }
        media_IRI = MediaIoBaseUpload(buffer_IRI, mimetype="application/vnd.openxmlformats-officedocument.spreadsheetml.sheet")
        drive_service.files().create(
            body=file_metadata_IRI,
            media_body=media_IRI,
            fields="id",
            supportsAllDrives=True
        ).execute()

        logging.info(f"Fichier IRI généré : IRI_CRP_{sem}.xlsx")

        # Enregistrement de DailySCC_rouge
        buffer_rouge = BytesIO()
        fichier_weekly_Rouge.to_excel(buffer_rouge, index=False)
        buffer_rouge.seek(0)

        file_metadata_rouge = {
            "name": f"DailySCC_IG_{sem}.xlsx",
            "parents": [folder['id']],  
            "mimeType": "application/vnd.openxmlformats-officedocument.spreadsheetml.sheet"
        }

        media_rouge = MediaIoBaseUpload(buffer_rouge, mimetype="application/vnd.openxmlformats-officedocument.spreadsheetml.sheet")

        drive_service.files().create(
            body=file_metadata_rouge,
            media_body=media_rouge,
            fields="id",
            supportsAllDrives=True
        ).execute()

        logging.info(f"Fichier DailySCC_rouge généré : DailySCC_IG{sem}.xlsx")

        #  Enregistrement de DailySCC_carma
        buffer_orange = BytesIO()
        fichier_weekly_Orange.to_excel(buffer_orange, index=False)
        buffer_orange.seek(0)

        file_metadata_orange = {
            "name": f"DailySCC_CARMA_{sem}.xlsx",
            "parents": [folder['id']], 
            "mimeType": "application/vnd.openxmlformats-officedocument.spreadsheetml.sheet"
        }

        media_orange = MediaIoBaseUpload(buffer_orange, mimetype="application/vnd.openxmlformats-officedocument.spreadsheetml.sheet")

        drive_service.files().create(
            body=file_metadata_orange,
            media_body=media_orange,
            fields="id",
            supportsAllDrives=True
        ).execute()

        
        logging.info(f"Fichier DailySCC_CARMA généré : DailySCC_carma{sem}.xlsx")

        #  Enregistrement de DailySCC_CB
        buffer_CB = BytesIO()
        weekly_CB.to_excel(buffer_CB, index=False)
        buffer_CB.seek(0)

        file_metadata_CB = {
            "name": f"DailySCC_CB_{sem}.xlsx",
            "parents": [folder['id']], 
            "mimeType": "application/vnd.openxmlformats-officedocument.spreadsheetml.sheet"
        }

        media_CB = MediaIoBaseUpload(buffer_CB, mimetype="application/vnd.openxmlformats-officedocument.spreadsheetml.sheet")

        drive_service.files().create(
            body=file_metadata_CB,
            media_body=media_CB,
            fields="id",
            supportsAllDrives=True
        ).execute()

        
        logging.info(f"Fichier DailySCC_CB généré : DailySCC_CB{sem}.xlsx")


        #  Enregistrement de DailySCC_HYPER
        buffer_hyper = BytesIO()
        weekly_hyper.to_excel(buffer_hyper, index=False)
        buffer_hyper.seek(0)

        file_metadata_hyper = {
            "name": f"DailySCC_hyper_{sem}.xlsx",
            "parents": [folder['id']], 
            "mimeType": "application/vnd.openxmlformats-officedocument.spreadsheetml.sheet"
        }

        media_hyper = MediaIoBaseUpload(buffer_hyper, mimetype="application/vnd.openxmlformats-officedocument.spreadsheetml.sheet")

        drive_service.files().create(
            body=file_metadata_hyper,
            media_body=media_hyper,
            fields="id",
            supportsAllDrives=True
        ).execute()

        
        logging.info(f"Fichier DailySCC_hyper généré : DailySCC_hyper{sem}.xlsx")


            
    except Exception as e:
        logging.error(f"Erreur lors du traitement du fichier {stream.name} (message ID: {msg_id}) : {e}")


if new_ids:
    today_str = datetime.now().strftime("%Y/%m/%d")
    try:
        save_state(drive_service, file_id, ids, today_str)
        logging.info(f"État mis à jour avec {len(new_ids)} nouveaux IDs et nouvelle date : {today_str}")
    except Exception as e:
        logging.error(f"Erreur lors de la sauvegarde de l'état : {e}")


logging.info("Fin du traitement.")

